In [ ]:
"""
Experiments with dataset in paper,
Explainable Verbal Deception Detection using Transformers
Loukas Ilias, Felix Soldner and Bennett Kleinberg
uses LIWC-15
"""
import openai
import os

openai.api_key = os.environ["OPENAI_API_KEY"]  # source the ~/.zshrc file

# https://platform.openai.com/docs/guides/rate-limits/error-mitigation
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)  # for exponential backoff

# constants, until you change them ;-)
new_line = '\n'
nb_test_samples = 10
nb_few_shot_samples_of_each_class =3 # truth and deception
delimiter = '```\n'
# MODEL = "gpt-3.5-turbo"
# MODEL = "text-davinci-003"
MODEL = "gpt-4"


#### Get the dataset

In [ ]:
import pandas as pd
df = pd.read_csv ('LIWC-15 Results - sign_events_data_statements - LIWC Analysis')
# simple EDA
# print(df)
# print(df.columns)
print(f'shape: {df.shape}')  # should be 1640 x 6
df.head


#### Get the LIWC markers

In [ ]:

liwc_15 = pd.read_csv ('LIWC-15 Results - sign_events_data_statements - LIWC Analysis')
# simple EDA

print(f'shape: {liwc_15.shape}')  # should be 1640, 
print(liwc_15.columns)
cols = sorted(liwc_15.columns)
nb_attrib_per_line = 10
print_buf = []
for i, attrib in enumerate(cols):
    print_buf.append(attrib)
    if i % nb_attrib_per_line == 0:
        print(print_buf)
        print_buf = []

truth_markers = df[['ingest', 'bio', 'Analytic', 'number', 'leisure', 'focusfuture']]
deception_markers = df[['Apostro', 'focuspast', 'reward', 'WC', 'pronoun', 'ppron', 'Exclam', 'Tone']]
liwc_markers = df[['ingest', 'bio', 'Analytic', 'number', 'leisure', 'Apostro', 'focuspast', 'reward', 'WC', 'pronoun']]
# print(truth_markers)
# print(deception_markers)
print(liwc_markers)

#### Some quick test to see how the truth/deceit markers are bahaving

In [ ]:
import textwrap
import json
# TODO see if this can be more friendlier if GPT does not know about LIWC
def construct_liwc_attributes_json(row):
    # print('class:', 'truthful' if df.iloc[row]['outcome_class'] == 't' else 'deceptive')
    # print(liwc_markers.iloc[row])
    # print(f'q1:\n {textwrap.fill(df.iloc[row]["q1"], 100)}')
    # print(f'q2:\n {textwrap.fill(df.iloc[row]["q2"], 100)}')
    
    attributes = ['ingest', 'bio', 'Analytic', 'number', 'leisure', 'Apostro', 'focuspast', 'reward', 'WC', 'pronoun']
    data = {}
    for attribute in attributes:
        data[attribute] = str(row[attribute])
    
    return json.dumps(data)

print(df.iloc[0])
print(construct_liwc_attributes_json(df.iloc[0].copy()))

In [ ]:
def filter_by_class(df, category):
   return df[df['outcome_class']== category]

truth_df = filter_by_class(df, 't')
# print(truth_df)
print(f'truth df shape: {truth_df.shape}')  # should be 1640 x 6

# replace with a more expressive word, truthful
truth_df['outcome_class'] = df['outcome_class'].replace('t','truthful')
print(truth_df)

deceit_df = filter_by_class(df, 'd')
# print(deceit_df)
print(f'deceit df shape: {deceit_df.shape}')  # should be 1640 x 6

# replace with a more expressive word, deceitful
deceit_df['outcome_class'] = df['outcome_class'].replace('d','deceptive')
print(deceit_df)

# pick random non-repeating rows
def pick_randon_non_repeating(df, quantity):
    import random
    rand_df = pd.DataFrame()
    random_list = random.sample(range(df.shape[0]), quantity)
    print("non-repeating random numbers are:")
    return df.iloc[random_list], random_list

random_truth_df, truth_indices_list = pick_randon_non_repeating(truth_df, nb_few_shot_samples_of_each_class)
print(f'random truth list:\n, {random_truth_df}')
print(f'truth indices:" {truth_indices_list}')

random_deceit_df, deceit_indices_list = pick_randon_non_repeating(deceit_df, nb_few_shot_samples_of_each_class)
print(f'random deceit list:\n, {random_deceit_df}')
deceit_indices_list = [x + truth_df.shape[0] for x in deceit_indices_list] # do this to exclude from poriginal list
print(f'deceit indices: {deceit_indices_list}')

random_truth_deceit_df = pd.concat([random_truth_df, random_deceit_df])
few_shot_list = truth_indices_list + deceit_indices_list
print(f'truth + deceit indices" {few_shot_list}')



#### Setup the OpenAI call

In [ ]:
def get_chat_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def get_chat_completion_with_backoff(prompt, model="gpt-3.5-turbe"):
    return get_chat_completion(prompt, model)

def get_completion(prompt, model='text-davinci-003'):
    response = openai.Completion.create(
        prompt=prompt,
        model=model
    )
    return response.choices[0]['text']

In [ ]:
intro = f"""
You are tasked to classify the response to questions into two classes: truthful or deceptive.
You'll be presented with the following pieces of information on an activity:
(1) The title of the activity.
(2) An answwer to a question: Please describe your activity. Be as specific as possible.
(3) A answer to a follow-on question: What information can you give us to reassure us that you are telling the truth?
(4) A Linguistic Inquiry Word Count (LIWC) category values as JSON data
Using all four pieces of information, complete the response with either 'truthful' or 'deceptive'.

Here are a few examples delimited by triple backticks:

"""

response_1_heading = """Response #1:\n"""
response_2_heading = """Response #2:\n"""

liwc_header = """Linguistic Inquiry Word Count (LIWC):\n"""

def construct_activity_scenario(row):
    # activity_header = 'Title of the Activity: ' + new_line
    activity_header = 'Activity: ' + new_line
    activity_description_header = 'Question #1: \nPlease describe your activity. Be as specific as possible.'

    activity_reassurance_header = 'Question #2: \nWhat information can you give us to reassure us that you are telling the truth?'

    activity = activity_header + row['signevent'] + new_line
    q1 = activity_description_header + new_line + response_1_heading + row['q1'] + new_line
    q2 = activity_reassurance_header + new_line + response_2_heading + row['q2'] + new_line
    return activity + q1 + q2

def construct_outcome(row):
    outcome = "Is the response truthful or deceptive?\n"
    return outcome  + row['outcome_class'] + new_line

def construct_liwc_json(row):
    pass

def construct_few_shot_prompt(few_shot_df, infer_row):
    # constructed as a list
    prompt = []
    prompt.append(intro)
    
    for _, row in few_shot_df.iterrows():
        prompt.append(delimiter)
        prompt.append(construct_activity_scenario(row))
        prompt.append(liwc_header)
        prompt.append(construct_liwc_attributes_json(row))
        prompt.append(delimiter)
        prompt.append(construct_outcome(row))
        prompt.append(delimiter)
        prompt.append(new_line)    
        prompt.append(new_line)

    prompt.append(delimiter)
    prompt.append(construct_activity_scenario(infer_row))
    prompt.append(liwc_header)
    prompt.append(construct_liwc_attributes_json(row))
    prompt.append(delimiter) 
    prompt.append(construct_outcome(infer_row)) # has to have a blank outcome to be filled by the llm
    prompt.append(delimiter)


    return prompt



In [ ]:
def create_test_indices(df, total, exclude_list):
    import random
    rand_list = []
    count = 0
    while count < total:
        rand_row = random.randrange(df.shape[0])
        if rand_row not in exclude_list:
            rand_list.append(rand_row)
            count += 1
    return rand_list

In [133]:
test_indices = create_test_indices(df, nb_test_samples, few_shot_list)  # exclude the ones in the few shot list
# test_indices = [1435]
print(f'test indices: {test_indices}')


test indices: [746, 1619, 579, 824, 1241, 869, 476, 1167, 662, 1456]


In [134]:
y_ground_truth = []  # for computing F1-score
y_predicted = []

for index in test_indices:
    infer_row = df.loc[index].copy()
    # print(f'Inferring the `class_outcome` for:\n{infer_row}')
    ground_truth = 'truthful' if infer_row['outcome_class'] == 't' else 'deceptive'
    # mask the `outcome_class` field since you want to predict it
    infer_row['outcome_class'] = ''

    # print(f'Original\n:{df.loc[index]}')
    # print(f'infer row\n: {infer_row}')

    prompt = construct_few_shot_prompt(random_truth_deceit_df, infer_row)
    prompt = ''.join(prompt)
    
    # print(f'Prompt:\n{prompt}')

    response = get_chat_completion_with_backoff(
        prompt=prompt,
        model=MODEL,
    )    
        
    print(f'INDEX: {index} GROUND TRUTH: {ground_truth}, RESPONSE: {response} - {"wrong" if ground_truth != response else "correct"}')
    y_ground_truth.append(ground_truth)
    y_predicted.append(response)


INDEX: 746 GROUND TRUTH: truthful, RESPONSE: deceptive - wrong
INDEX: 1619 GROUND TRUTH: deceptive, RESPONSE: deceptive - correct
INDEX: 579 GROUND TRUTH: truthful, RESPONSE: deceptive - wrong
INDEX: 824 GROUND TRUTH: deceptive, RESPONSE: deceptive - correct
INDEX: 1241 GROUND TRUTH: deceptive, RESPONSE: deceptive - correct
INDEX: 869 GROUND TRUTH: deceptive, RESPONSE: truthful - wrong
INDEX: 476 GROUND TRUTH: truthful, RESPONSE: deceptive - wrong
INDEX: 1167 GROUND TRUTH: deceptive, RESPONSE: deceptive - correct
INDEX: 662 GROUND TRUTH: truthful, RESPONSE: deceptive - wrong
INDEX: 1456 GROUND TRUTH: deceptive, RESPONSE: deceptive - correct


### Compute metrics

In [135]:
from sklearn.metrics import f1_score
print('Weighted F1-score:', f1_score(y_ground_truth, y_predicted, average='weighted'))

Weighted F1-score: 0.4
